<img src="https://raw.githubusercontent.com/william-mx/foxflow/main/docs/foxflow_logo.png" width="360">

**FoxFlow** lets you load **Foxglove Cloud recordings** directly into **Jupyter or Google Colab** as pandas DataFrames.

It removes ROS setup overhead and makes sensor data easy to explore, visualize, export, and synchronize — all in pure Python.


## Setup

In [1]:
# Install foxflow
!pip install -qq foxflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.0 MB/s eta 0:00:00


In [ ]:
!pip uninstall -y -qq foxflow

In [1]:
!pip install -qq git+https://github.com/william-mx/foxflow.git@dev

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.8 MB/s eta 0:00:00


In [10]:
# Core foxflow reader
from foxflow.reader import BagfileReader

# Data handling
import pandas as pd
import numpy as np
import os

# Visualization
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Access Colab user secrets
from google.colab import userdata

# Read Foxglove API key
FOXGLOVE_KEY = userdata.get("FOXGLOVE_KEY")

In [4]:
# Create reader instance using your API key
r = BagfileReader(FOXGLOVE_KEY)

# List available Foxglove recordings
r.print_recordings()

Available Recordings:

bshaped_track_all                        → 2y57fL95RUnUG5MW
bc_acc_right_lidar_0                     → rec_0dfuKVP9v8D2kJiN
bc_acc_left_lidar_0                      → rec_0dfuKkRmcCvQjeS1
bc_left_vol_II_0                         → rec_0dfYV91ylwPKOGKf
bc_left                                  → rec_0dfT9OhmOBqhm5OL
bc_right                                 → rec_0dfT9VBIqKuqlW80
bc_right_vol_II_0                        → rec_0dfYVP76gVi5Rh40
bc_left_vol_III_0                        → rec_0dfpgCwFVqTB0N0P
bc_right_vol_III_0                       → rec_0dfpgQHkm8x9qGEz
bc_right_vol_IV_0                        → rec_0dgKHXBIK5BkyHiX
bc_left_vol_IV_0                         → rec_0dgKLXyLBbqJBzl9
bc_odometry_0                            → rec_0dvED5yH5a997zrp
bc_right_0                               → rec_0dvKbLpFyZmccUkf
bc_left_0                                → rec_0dvKbY45GilgZiTF
parking_spot_scan_boxes_mcap_0           → rec_0dXGZgmNC3UmsFgo


In [30]:
# List supported message types
r.list_available_message_types()

Supported message types:

ackermann_msgs/AckermannDrive
ackermann_msgs/AckermannDriveStamped
geometry_msgs/TransformStamped
sensor_msgs/CompressedImage
sensor_msgs/Image
sensor_msgs/Imu
sensor_msgs/Joy
sensor_msgs/LaserScan
sensor_msgs/MagneticField
sensor_msgs/NavSatFix
sensor_msgs/PointCloud2
std_msgs/Bool
std_msgs/Byte
std_msgs/ByteMultiArray
std_msgs/Char
std_msgs/Float32
std_msgs/Float32MultiArray
std_msgs/Float64
std_msgs/Float64MultiArray
std_msgs/Int16
std_msgs/Int16MultiArray
std_msgs/Int32
std_msgs/Int32MultiArray
std_msgs/Int64
std_msgs/Int64MultiArray
std_msgs/Int8
std_msgs/Int8MultiArray
std_msgs/String
std_msgs/UInt16
std_msgs/UInt16MultiArray
std_msgs/UInt32
std_msgs/UInt32MultiArray
std_msgs/UInt64
std_msgs/UInt64MultiArray
std_msgs/UInt8
std_msgs/UInt8MultiArray
tf2_msgs/TFMessage


## Choose a Recording

In [5]:
# Select a recording by its name
df_info = r.select_recording_by_name("bshaped_track_all")

# Show basic information about the selected recording
display(df_info)

,topic,version,encoding,schema_encoding,schema_name
0,/camera/color/image_jpeg,b468820764e8c96db6b16dc56c40ee82,ros1,ros1msg,sensor_msgs/CompressedImage
1,/commands/motor/brake,fdb28210bfa9d7c91146260178d9a584,ros1,ros1msg,std_msgs/Float64
2,/commands/motor/speed,fdb28210bfa9d7c91146260178d9a584,ros1,ros1msg,std_msgs/Float64
3,/commands/servo/position,fdb28210bfa9d7c91146260178d9a584,ros1,ros1msg,std_msgs/Float64
4,/imu,123d1a9d41bc3f05fc79bb94eea854a2,ros1,ros1msg,std_msgs/Float32MultiArray
5,/imu_calibrated,b881254029ebe36c32bb3458f9cf7514,ros1,ros1msg,sensor_msgs/Imu
6,/imu_filtered,b881254029ebe36c32bb3458f9cf7514,ros1,ros1msg,sensor_msgs/Imu
7,/rc/ackermann_cmd,644e625bcce4f0c6306852640561230a,ros1,ros1msg,ackermann_msgs/AckermannDriveStamped
8,/rc/joy,290607a844607e9731b73ae070905527,ros1,ros1msg,sensor_msgs/Joy
9,/rosout,bb11168e13f5747d99862b114db8fa57,ros1,ros1msg,rosgraph_msgs/Log


## Reading Sensor Data

#### LaserScan

In [14]:
# Load scan data into a DataFrame
df_scan = r.read_topic("/scan")

# Inspect the data
df_scan.head()

,timestamp_ns,angle_min,angle_max,angle_increment,time_increment,scan_time,range_min,range_max,ranges,intensities
0,1724765785919214824,-3.124139,3.141593,0.008715,0.000102,0.073493,0.15,16.0,"(inf, inf, inf, inf, 3.635999917984009, 3.6359...","(0.0, 0.0, 0.0, 0.0, 47.0, 47.0, 0.0, 0.0, 47...."
1,1724765785993059912,-3.124139,3.141593,0.008715,0.000119,0.085666,0.15,16.0,"(inf, inf, inf, inf, inf, 3.635999917984009, 3...","(0.0, 0.0, 0.0, 0.0, 0.0, 47.0, 47.0, 0.0, 47...."
2,1724765786079006888,-3.124139,3.141593,0.008715,0.000101,0.072932,0.15,16.0,"(inf, inf, inf, inf, inf, inf, inf, 3.64000010...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47.0, 47.0..."
3,1724765786152232584,-3.124139,3.141593,0.008715,0.000102,0.073639,0.15,16.0,"(inf, inf, inf, inf, inf, 3.6559998989105225, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 47.0, 47.0, 47.0, 47..."
4,1724765786226266184,-3.124139,3.141593,0.008715,0.000102,0.073566,0.15,16.0,"(inf, inf, inf, inf, inf, inf, 3.6440000534057...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 47.0, 47.0, 47...."


In [15]:
# Take one scan frame
row = df_scan.iloc[0]

# Extract distance measurements
ranges = np.array(row["ranges"])

# Create angle values (radians → degrees)
angles = np.degrees(
    np.linspace(
        row["angle_min"],
        row["angle_max"],
        num=len(ranges),
        endpoint=False,
    )
)

# Plot the scan as a polar scatter plot
fig = px.scatter_polar(
    r=ranges,
    theta=angles,
    color=ranges,  # distance encoded as color
    labels={"color": "range [m]"},
)

fig.show()

#### Image

In [6]:
# Read camera frames and metadata
df_cam, images = r.read_topic("/camera/color/image_jpeg", return_images=True)

# Inspect the first entries
df_cam.head()

,timestamp_ns
0,1724765785988855123
1,1724765786022517442
2,1724765786055614233
3,1724765786091928005
4,1724765786122303724


In [9]:
# Display the first camera image
fig = px.imshow(images[0])
fig.show()

In [13]:
# Directory where images will be saved
export_dir = "export"

# Read camera data and export images to disk
df_cam = r.read_topic(
    "/camera/color/image_jpeg",
    export=True,
    export_dir=export_dir,
)

# List a few exported images to verify the export
os.listdir(export_dir)[:5]

['1724765786555867910.jpg',
 '1724765818512644052.jpg',
 '1724765791659696578.jpg',
 '1724765808706725835.jpg',
 '1724765804935825824.jpg']

#### NavSatFix

In [ ]:
# Select the recording to work with
df_info = r.select_recording_by_name("rtb")

# Load GPS (NavSatFix) data into a DataFrame
df_sat = r.read_topic("/sensor_stack/adma/fix")

# Plot the driven route on a map
fig = px.scatter_geo(
    df_sat,
    lat="latitude_deg",
    lon="longitude_deg",
)

fig.show()

#### PointCloud2

In [ ]:
# Select the recording to work with
df_info = r.select_recording_by_name("rtb")

# Load one LiDAR topic into a DataFrame
df_pc = r.read_topic("/sensor_stack/lidars/lid_1/livox/lidar_192_168_1_150")

# Extract the first point cloud frame (x, y, z, intensity)
points = df_pc.loc[0, "points"]

# Visualize the point cloud in 3D
fig = go.Figure(
    go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode="markers",
        marker=dict(
            size=2,
            color=points[:, 3],  # intensity → color
            colorscale="Viridis",
            showscale=True,
        ),
    )
)

fig.show()

## Working with Events

#### Event Timeline Overview

In [20]:
df_info = r.select_recording_by_name("bc_left")

events = r.get_events()

# Flatten nested dicts into a table
df = pd.json_normalize(events)

# Simple timeline: one bar per event
fig = px.timeline(
    df,
    x_start="start",   # event start time
    x_end="end",       # event end time
    y="id",            # one row per event
    hover_data=["id"], # minimal hover info
)

# Display plot
fig.show()

#### Accessing Data Within Events

In [ ]:
# Read events for a topic
events_dict = r.read_events("/camera/camera/color/image_raw", return_images=True)

# events_dict = {event_id: {"event", "df", "images"}}

In [32]:
# Show all available event IDs
print(events_dict.keys())

dict_keys(['evt_0dgHvEzdDs4USi8q', 'evt_0dgHv8YOA5AaSswn', 'evt_0dgHuyDQoZQx5cvJ', 'evt_0dgHuoVwCvzzU8hi', 'evt_0dgHucb06Q3HDXXG', 'evt_0dgHuX7tUUuncEYi', 'evt_0dgHuNjKaY3WHCo3'])


**What this returns**

`events` is a **dictionary of events**, indexed by event ID:

```text
{
  event_id → {
    "event"  : event metadata
    "df"     : DataFrame with data during the event
    "images" : list of images (if return_images=True)
  }
}
```



In [30]:
# Select the first available event
event = next(iter(events_dict.values()))

# Read event metadata
metadata = event["event"]["metadata"]

# Take the first 5 images of this event
imgs = event["images"][:5]

# Create a readable title from metadata key:value pairs
title = ", ".join(f"{k}: {v}" for k, v in metadata.items())

# Plot images side by side
fig = px.imshow(
    np.stack(imgs),
    facet_col=0,
    title=title,
)

fig.show()


## Synchronizing Data

In [35]:
from foxflow.utils import sync_dataframes

In [34]:
# Select the recording to work with
df_info = r.select_recording_by_name("bshaped_track_all")

# Load camera data and images
df_cam, images = r.read_topic("/camera/color/image_jpeg", return_images=True)

# Load steering command data
df_cmd = r.read_topic("/rc/ackermann_cmd")

# Load IMU data (gyro, accel)
df_imu = r.read_topic("/imu_calibrated")

# Synchronize all DataFrames by time
df_cam, df_cmd, df_imu = sync_dataframes(df_cam, df_cmd, df_imu)

In [44]:
# Plot steering angle and gyro z after synchronization
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        y=df_cmd["steering_angle"],
        name="Steering angle",
    )
)

fig.add_trace(
    go.Scatter(
        y=-df_imu["gyro_z"],
        name="Gyro Z",
        yaxis="y2",
    )
)

fig.update_layout(
    yaxis=dict(title="Steering angle"),
    yaxis2=dict(
        title="Gyro Z",
        overlaying="y",
        side="right",
    ),
)

fig.show()


## Data Export

In [ ]:
# Directory where images will be saved
export_dir = "export"

# Select the recording to work with
df_info = r.select_recording_by_name("bshaped_track_all")

# Parse camera topic and export images
ex_dir = r.export_bagfile(export_dir, df_format='csv')